<a href="https://colab.research.google.com/github/shahriarivari/Persian_sentiment_analysis/blob/main/BERT_notebooks/Pretrain_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pip installs

In [ ]:
!pip install tokenizers
!pip install datasets
!pip install -U accelerate
!pip install -U transformers

#import libraries

In [ ]:
import os
from transformers import BertTokenizerFast
from transformers import BertConfig , BertForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from transformers import TrainerCallback
import logging

##import dataset

In [ ]:
# You should just change this part in order to download your
# parts of corpus.
# There are a total of 126 (indexing from 0-> 125) files 0f training text each with size of 1.07 GB roughly
# There are a total of 3 (indexing from 0-> 2) files of testing text each with size of 1.07 GB roughly 
# for more cehck the link -> https://huggingface.co/datasets/SLPL/naab

indices = {
    "train": [81, 14, 3,
              # 94, 35,
              # 41, 28, 67, 55, 79
              ],
    "test": [0,
            #  1
             ]
}

N_FILES = {
    "train": 126,
    "test": 3
}
_BASE_URL = "https://huggingface.co/datasets/SLPL/naab/resolve/main/data/"
data_url = {
    "train": [_BASE_URL + "train-{:05d}-of-{:05d}.txt".format(x, N_FILES["train"]) for x in range(N_FILES["train"])],
    "test": [_BASE_URL + "test-{:05d}-of-{:05d}.txt".format(x, N_FILES["test"]) for x in range(N_FILES["test"])],
}
for index in indices['train']:
    assert index < N_FILES['train']
for index in indices['test']:
    assert index < N_FILES['test']
data_files = {
    "train": [data_url['train'][i] for i in indices['train']],
    "test": [data_url['test'][i] for i in indices['test']]
}

In [ ]:
dataset = load_dataset('text', data_files=data_files, use_auth_token=False)

#set paths and file names


In [ ]:
# Set your paths and file names
tokenizer_output_dir = "bert_tokenizer"
os.mkdir(tokenizer_output_dir)
pretrained_model_output_dir = "bert_pretrained_model"
os.mkdir(pretrained_model_output_dir)

##load tokenzier

To load the tokenizer, after creating the folder, copy and paste the "vocab" file given after saving the tokenizer
and then use the line below

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(tokenizer_output_dir,max_length= 256)

##tokenize the dataset

In [ ]:
def batch_tokenize_function(batch):
    # Tokenize the text using BertTokenizerFast
    encoded = tokenizer(batch["text"], truncation=True, padding='max_length',
                        return_special_tokens_mask=True,
                        max_length= 256, return_tensors='pt')

    return {
        'input_ids': encoded['input_ids'],
        'attention_mask': encoded['attention_mask'],
    }


tokenized_train_dataset = dataset['train'].map(batch_tokenize_function, batched=True)
tokenized_test_dataset = dataset['test'].map(batch_tokenize_function, batched=True)

tokenized_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
tokenized_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

##BERT model configuration

In [ ]:
vocab_size = 10_000    # same as tokenizer
max_length = 256       # same as toeknizer

# Model Configuration
model_config = BertConfig(
    vocab_size=vocab_size,       # the voacb size of the Bert base is 30_522
    hidden_size=64,              # Adjust as needed the BERT base has hidden_size 0f 768
    num_attention_heads=8,       # Adjust as needed the BERT base has 12 attention heads
    num_hidden_layers=4,         # Adjust as needed the BERT base has 12 hidden layers 
    max_position_embeddings=max_length,
)

# Model Initialization
model = BertForMaskedLM(config=model_config)

##set the data collator

In [ ]:
# using data collator for masking tokens and passing them to the model 

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
    )

##training arguments

In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir=pretrained_model_output_dir, # output directory to where save model checkpoint
    overwrite_output_dir=True,
    num_train_epochs=1,                     # number of training epochs, feel free to tweak
    per_device_train_batch_size=64,         # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=100,        # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,          # evaluation batch size
    logging_steps=1000,                     # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    save_total_limit=3,                     # whether you don't have much space so you let only 3 model weights saved in the disk
    evaluation_strategy="steps",            # evaluate each `logging_steps` steps
    eval_steps=1000,
)

##train the model

In [ ]:
# Trainer Initialization
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train_dataset,
    eval_dataset = tokenized_test_dataset,
    data_collator = data_collator,
)

# Train the model
trainer.train()

# Save the final pre-trained model
trainer.save_model(os.path.join(pretrained_model_output_dir, "final_model"))

##saving the model in google drive

In [ ]:
import shutil
#First zip the model_dir then move it to google drive

source_dir = '/content/bert_pretrained_model'       # assuming you are using google colab, feel free to change this directory acording to your code
output_filename = pretrained_model_output_dir

# Create a Zip file
shutil.make_archive(output_filename, 'zip', source_dir)

In [ ]:
# mount google  drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Move the zip file to Google Drive
drive_path = '/content/drive/MyDrive/'
output_filename = f'/content/{output_filename}.zip'
shutil.move(output_filename, drive_path)